In [1]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt

In [2]:
df = pd.read_csv('results2.csv')

In [3]:
df2 = df.copy()

In [4]:
df2

,persona,text,likert_scale,Itemnum,answer
0,Saddam Hussein,Is talkative,1 to 5,itemcount,5
1,Saddam Hussein,Tends to find fault with others,1 to 5,itemcount,4
2,Saddam Hussein,"Is depressed, blue",1 to 5,itemcount,5
3,Saddam Hussein,"Is original, comes up with new ideas",1 to 5,itemcount,4
4,Saddam Hussein,"Is relaxed, handles stress well",1 to 5,itemcount,5
...,...,...,...,...,...
2592,Ted Kaczynski,"Likes to reflect, play with ideas",1 to 5,40,4
2593,Ted Kaczynski,Has few artistic interests,1 to 5,41,3
2594,Ted Kaczynski,Likes to cooperate with others,1 to 5,42,2
2595,Ted Kaczynski,Is easily distracted,1 to 5,43,5


In [5]:
df3=df2[1629:2598]

In [6]:
df3

,persona,text,likert_scale,Itemnum,answer
1629,Saddam Hussein,Is talkative,1 to 5,1,5
1630,Saddam Hussein,Tends to find fault with others,1 to 5,2,5
1631,Saddam Hussein,Does a thorough job,1 to 5,3,3
1632,Saddam Hussein,"Is depressed, blue",1 to 5,4,1
1633,Saddam Hussein,"Is original, comes up with new ideas",1 to 5,5,4
...,...,...,...,...,...
2592,Ted Kaczynski,"Likes to reflect, play with ideas",1 to 5,40,4
2593,Ted Kaczynski,Has few artistic interests,1 to 5,41,3
2594,Ted Kaczynski,Likes to cooperate with others,1 to 5,42,2
2595,Ted Kaczynski,Is easily distracted,1 to 5,43,5


In [7]:
print(df3['Itemnum'].dtype)  # Should show 'object'

# Attempt to convert 'itemnum' to integer
try:
    df3['Itemnum'] = df3['Itemnum'].astype(int)
except ValueError:
    # If there's an error, it might be due to non-numeric values
    print("Conversion failed due to non-numeric values.")
    
    # Convert 'itemnum' to numeric, setting errors='coerce' will replace non-numeric values with NaN
    df3['Itemnum'] = pd.to_numeric(df3['Itemnum'], errors='coerce')

    # Check for rows where conversion to int failed
    print(df3[df3['Itemnum'].isna()])

    # Optionally drop or handle NaN values
    # data.dropna(subset=['itemnum'], inplace=True)

# Check the data type after conversion
print(df3['Itemnum'].dtype)

object
int64


C:\Users\PC\AppData\Local\Temp\ipykernel_4156\4226010020.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Itemnum'] = df3['Itemnum'].astype(int)


In [8]:
df3.dtypes

persona         object
text            object
likert_scale    object
Itemnum          int64
answer           int64
dtype: object

In [10]:
import pandas as pd
import numpy as np

# Read the CSV file
df=df3.copy()
# Define the scoring keys
scoring_key = {
    'Extraversion': [1, 6, 11, 16, 21, 26, 31, 36],
    'Agreeableness': [2, 7, 12, 17, 22, 27, 32, 37, 42],
    'Conscientiousness': [3, 8, 13, 18, 23, 28, 33, 38, 43],
    'Neuroticism': [4, 9, 14, 19, 24, 29, 34, 39],
    'Openness': [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]
}

# Define reverse-scored items
reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]
reverse_scored = [6, 21, 31, 2, 12, 27, 37, 8, 18, 23, 43, 9, 24, 34, 35, 41]

# Function to adjust scores
def adjust_score(row):
    #print("for this row", row['persona'],"$$$$$", row['Itemnum'])
    if row['Itemnum'] in reverse_scored:
        #print("hit")
        return 6 - row['answer']
    return row['answer']

# Add adjusted score column
df['adjusted_score'] = df.apply(adjust_score,axis=1)
#print(df[:44])
# Function to calculate trait score
def calculate_trait_score(persona_data, trait_items):
    scores = []
    for item in trait_items:
        #print("§§§§§§§§§§§§§§§§§§§§")
        score = persona_data.loc[persona_data['Itemnum'] == item, 'answer'].iloc[0]
        if item in reverse_scored:
            score = 6 - score  # Reverse the score
        scores.append(score)
    return np.mean(scores)

# Group by persona and calculate scores
results = []
for persona, group in df.groupby('persona'):
    scores = {}
    for trait, items in scoring_key.items():
        scores[trait] = calculate_trait_score(group, items)
    results.append({'Persona': persona, **scores})

# Create a dataframe with the results
results_df = pd.DataFrame(results)

# Display the results
print(results_df)
results_df.to_csv('bfi_scores.csv', index=False)

                 Persona  Extraversion  Agreeableness  Conscientiousness  \
0           Adolf Hitler         3.000       2.777778           2.555556   
1              Al Capone         2.875       2.666667           3.777778   
2       Benito Mussolini         3.750       2.888889           3.000000   
3          Bonnie Parker         2.750       2.777778           2.666667   
4         Charles Manson         2.375       2.666667           3.777778   
5           Fidel Castro         2.500       3.000000           3.222222   
6         Harold Shipman         2.875       3.333333           3.000000   
7               Idi Amin         3.375       2.555556           3.111111   
8         Jeffrey Dahmer         3.000       3.444444           3.333333   
9        John Wayne Gacy         3.625       3.111111           2.444444   
10          Josef Stalin         3.500       2.888889           3.111111   
11           Kim Il Sung         3.250       2.333333           4.555556   
12          

In [11]:
results_df.round()

Persona               object
Extraversion         float64
Agreeableness        float64
Conscientiousness    float64
Neuroticism          float64
Openness             float64
dtype: object

In [78]:
# Check the shape of the dataframe
print(df.shape)
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check if we have 22 unique personas
print(df['persona'].nunique())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check if each persona has 44 items
print(df.groupby('persona')['Itemnum'].nunique().value_counts())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check the range of itemnum
print(df['Itemnum'].min(), df['Itemnum'].max())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Check the range of answer values
print(df['answer'].min(), df['answer'].max())
print("§§§§§§§§§§§§§§§§§§§§§§§§§§§")
# Display a sample of the data
print(df.head(10))

(968, 5)
§§§§§§§§§§§§§§§§§§§§§§§§§§§
22
§§§§§§§§§§§§§§§§§§§§§§§§§§§
Itemnum
44    22
Name: count, dtype: int64
§§§§§§§§§§§§§§§§§§§§§§§§§§§
1 9
§§§§§§§§§§§§§§§§§§§§§§§§§§§
1 6
§§§§§§§§§§§§§§§§§§§§§§§§§§§
             persona                                    text likert_scale  \
1629  Saddam Hussein                            Is talkative       1 to 5   
1630  Saddam Hussein         Tends to find fault with others       1 to 5   
1631  Saddam Hussein                     Does a thorough job       1 to 5   
1632  Saddam Hussein                      Is depressed, blue       1 to 5   
1633  Saddam Hussein    Is original, comes up with new ideas       1 to 5   
1634  Saddam Hussein                             Is reserved       1 to 5   
1635  Saddam Hussein    Is helpful and unselfish with others       1 to 5   
1636  Saddam Hussein                Can be somewhat careless       1 to 5   
1637  Saddam Hussein         Is relaxed, handles stress well       1 to 5   
1638  Saddam Hussein  Is cu

In [50]:
pd.set_option('display.max_rows', 500)

In [51]:


data = df3
# List of item numbers for each BFI trait and reverse scored items
extraversion_items = [1, 6, 11, 16, 21, 26, 31, 36]
agreeableness_items = [2, 7, 12, 17, 22, 27, 32, 37, 42]
conscientiousness_items = [3, 8, 13, 18, 23, 28, 33, 38, 43]
neuroticism_items = [4, 9, 14, 19, 24, 29, 34, 39]
openness_items = [5, 10, 15, 20, 25, 30, 35, 40, 41, 44]

reverse_scored_items = [6, 21, 31, 2, 12, 27, 37, 42, 8, 18, 23, 43, 9, 24, 34, 35, 41]

# Function to adjust for reverse scored items
def adjust_reverse_score(row):
    if row['Itemnum'] in reverse_scored_items:
        return 6 - row['answer']
        print("hit")
    else:
        return row['answer']


# Apply the reverse scoring adjustment
data['adjusted_score'] = data.apply(adjust_reverse_score, axis=1)
print(data[0:44])


             persona                                           text  \
1629  Saddam Hussein                                   Is talkative   
1630  Saddam Hussein                Tends to find fault with others   
1631  Saddam Hussein                            Does a thorough job   
1632  Saddam Hussein                             Is depressed, blue   
1633  Saddam Hussein           Is original, comes up with new ideas   
1634  Saddam Hussein                                    Is reserved   
1635  Saddam Hussein           Is helpful and unselfish with others   
1636  Saddam Hussein                       Can be somewhat careless   
1637  Saddam Hussein                Is relaxed, handles stress well   
1638  Saddam Hussein         Is curious about many different things   
1639  Saddam Hussein                              Is full of energy   
1640  Saddam Hussein                    Starts quarrels with others   
1641  Saddam Hussein                           Is a reliable worker   
1642  

C:\Users\PC\AppData\Local\Temp\ipykernel_10320\3711336549.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['adjusted_score'] = data.apply(adjust_reverse_score, axis=1)


In [62]:
(data['persona'][1])

KeyError: 1

In [63]:
type(data)

pandas.core.frame.DataFrame

In [36]:
# Creating a pivot table
pivot_table = data.pivot(index='persona', columns='Itemnum', values='adjusted_score')
print(pivot_table)

Itemnum               1  10  11  12  13  14  15  16  17  18  ...  40  41  42  \
persona                                                      ...               
Adolf Hitler          4   4   5   5   3   3   3   1   3   2  ...   3   4   5   
Al Capone             3   3   5   5   3   5   3   4   3   1  ...   5   3   2   
Benito Mussolini      3   4   3   3   4   3   3   5   3   3  ...   4   1   3   
Bonnie Parker         2   2   4   4   5   4   4   2   3   3  ...   3   5   3   
Charles Manson        2   4   2   5   4   1   2   5   4   5  ...   5   4   5   
Fidel Castro          4   1   1   3   5   3   3   3   3   4  ...   3   3   2   
Harold Shipman        3   1   1   1   5   4   3   2   5   4  ...   3   3   1   
Idi Amin              3   1   4   5   5   3   4   4   3   3  ...   3   3   1   
Jeffrey Dahmer        1   5   4   4   5   4   5   2   3   5  ...   2   5   5   
John Wayne Gacy       3   5   3   2   5   5   1   5   1   5  ...   5   5   4   
Josef Stalin          5   3   1   3   4 

In [37]:

# Summing scores for each BFI trait
pivot_table['Extraversion'] = pivot_table[extraversion_items].sum(axis=1)
pivot_table['Agreeableness'] = pivot_table[agreeableness_items].sum(axis=1)
pivot_table['Conscientiousness'] = pivot_table[conscientiousness_items].sum(axis=1)
pivot_table['Neuroticism'] = pivot_table[neuroticism_items].sum(axis=1)
pivot_table['Openness'] = pivot_table[openness_items].sum(axis=1)

# Selecting only the BFI scores columns
bfi_scores = pivot_table[['Extraversion', 'Agreeableness', 'Conscientiousness', 'Neuroticism', 'Openness']].reset_index()

# Display the BFI scores
print(bfi_scores)

KeyError: "None of [Index([1, 6, 11, 16, 21, 26, 31, 36], dtype='int64', name='Itemnum')] are in the [columns]"

In [17]:
ques_per_person = df3.groupby('persona').size().sort_values(ascending=False)
ques_per_person

persona
Adolf Hitler            44
Al Capone               44
Benito Mussolini        44
Bonnie Parker           44
Charles Manson          44
Fidel Castro            44
Harold Shipman          44
Idi Amin                44
Jeffrey Dahmer          44
John Wayne Gacy         44
Josef Stalin            44
Kim Il Sung             44
Mao Zedong              44
Muammar Gaddafi         44
Osama Bin Laden         44
Pablo Escobar           44
Pedro López             44
Saddam Hussein          44
Saloth "Pol Pot" Sar    44
Suharto                 44
Ted Bundy               44
Ted Kaczynski           44
dtype: int64

In [17]:
ques_per_person = df2.groupby('persona').size().sort_values(ascending=False)


In [18]:
ques_per_person

persona
Ted Kaczynski           36
John Wayne Gacy         36
Idi Amin                35
Saloth "Pol Pot" Sar    34
Suharto                 33
Harold Shipman          32
Jeffrey Dahmer          32
Al Capone               31
Pablo Escobar           31
Josef Stalin            31
Charles Manson          31
Fidel Castro            30
Bonnie Parker           30
Adolf Hitler            30
Mao Zedong              30
Ted Bundy               28
Saddam Hussein          28
Kim Il Sung             28
Pedro López             27
Osama Bin Laden         26
Benito Mussolini        23
Muammar Gaddafi         22
dtype: int64

In [19]:
ques_dist = df2.groupby('text').size().sort_values(ascending=False)
ques_dist

text
Has an assertive personality                     19
Remains calm in tense situations                 19
Starts quarrels with others                      19
Has few artistic interests                       18
Is sophisticated in art, music, or Literature    18
Is easily distracted                             18
Values artistic, aesthetic experiences           18
Is considerate and kind to almost everyone       17
Is talkative                                     17
Is full of energy                                17
Can be tense                                     17
Does a thorough job                              17
Tends to find fault with others                  16
Makes plans and follows through with them        16
Tends to be disorganized                         16
Is depressed, blue                               16
Is ingenious, a deep thinker                     16
Is reserved                                      16
Is helpful and unselfish with others             16
Generat